In [1]:
import pandas as pd
import re

In [2]:
s_loc = './data/sample/'
d_sets = ['Black-Male-Names.csv','Black-Female-Names.csv','Hispanic-Male-Names.csv','Hispanic-Female-Names.csv','White-Male-Names.csv','White-Female-Names.csv','Indian-Male-Names.csv','Indian-Female-Names.csv']
d_sets

['Black-Male-Names.csv',
 'Black-Female-Names.csv',
 'Hispanic-Male-Names.csv',
 'Hispanic-Female-Names.csv',
 'White-Male-Names.csv',
 'White-Female-Names.csv',
 'Indian-Male-Names.csv',
 'Indian-Female-Names.csv']

In [3]:
map_info = {'gender': {0:'Male',1:'Female'}, 'race' : {0:'black',1:'hispanic',2:'white',3:'indian'}}
map_info

{'gender': {0: 'Male', 1: 'Female'},
 'race': {0: 'black', 1: 'hispanic', 2: 'white', 3: 'indian'}}

In [4]:
data = pd.DataFrame([])

In [5]:
#rows_c = 1000
for i in d_sets:
    temp = pd.read_csv(s_loc+i)#,nrows=rows_c)
    data = data.append(temp)

In [6]:
data.head(10)

name  gender  race  length  fl_is_v  ll_is_v first_2 last_2 first_3  \
0        aaric       0     0       5        1        0      aa     ic     aar   
1    dominique       0     0       9        0        1      do     ue     dom   
2     fredrick       0     0       8        0        0      fr     ck     fre   
3       jarvis       0     0       6        0        0      ja     is     jar   
4      lorenzo       0     0       7        0        1      lo     zo     lor   
5         paul       0     0       4        0        0      pa     ul     pau   
6      richard       0     0       7        0        0      ri     rd     ric   
7        tyler       0     0       5        0        0      ty     er     tyl   
8  elsharnoubi       0     0      11        1        1      el     bi     els   
9       calvin       0     0       6        0        0      ca     in     cal   

  last_3  
0    ric  
1    que  
2    ick  
3    vis  
4    nzo  
5    aul  
6    ard  
7    ler  
8    ubi  
9    vin

In [220]:
data.dropna(inplace=True)

In [221]:
data.gender.value_counts()

0    27656
1    16192
Name: gender, dtype: int64

In [222]:
data.race.value_counts()

3    17712
0    13137
2     9063
1     3936
Name: race, dtype: int64

#### X & Y

In [223]:
data.columns

Index(['name', 'gender', 'race', 'length', 'fl_is_v', 'll_is_v', 'first_2',
       'last_2', 'first_3', 'last_3'],
      dtype='object')

In [224]:
c_list = ['length','ll_is_v','last_2','first_2']

In [225]:
X = data.iloc[:,3:]#data[c_list]

In [226]:
y = data.race.values
len(y)

43848

### Train-Test split

In [227]:
from sklearn.model_selection import train_test_split

In [228]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [229]:
pd.Series(y_test).value_counts()

3    3556
0    2575
2    1858
1     781
dtype: int64

### Utiities

In [19]:
def name_prep(word):
    #te = re.sub(r'/.*|@.*','',word).strip()
    te = str(word).lower()
    te = re.sub(r'[^a-z.\s].*','',te).strip()
    te = re.sub(r'[^a-z ]+','',te).strip()
    te = re.sub(r' +',' ',te).strip()
    te = re.sub(r'along.*|with.*|and.*',' ',te).strip()
    #te = re.sub(r'\b\w$|\b\w\w$','',te).strip()
    te = re.sub(r'\b\w\b|\b\w\w\b','',te).strip()
    #te = re.sub(r'\bna\b|\bNaN\b','',te).strip()
    if(len(te) < 3):
        return None
    else:
        return te

In [20]:
def conv_input(name,vectorizer):
    pre = name
    return vectorizer.transform([pre])

In [21]:
def predict_from_name(model,name,vectorizer):
    pre = name_prep(name)
    if(pre is None):
        print("Invalid !!!")
        return
    else:
        p = model.predict(conv_input(pre,vectorizer).toarray())
        print("Name : ",name,"\n","Race :",map_info['race'][p[0]])

### Character Vectorization

In [255]:
names = data.name

In [256]:
name_train, name_test, race_train, race_test = train_test_split(names,y,test_size=0.2)

In [22]:
def name_to_list(names):
    return list(map(lambda x: str(x),names))

In [257]:
names_list = name_to_list(name_train)

* Character count Vectorizer

In [258]:
from sklearn.feature_extraction.text import CountVectorizer

In [259]:
char_vec = CountVectorizer(lowercase=True,token_pattern='.')

In [260]:
char_name_vec = char_vec.fit_transform(names_list)

In [261]:
char_name_vec

<35078x27 sparse matrix of type '<class 'numpy.int64'>'
	with 222675 stored elements in Compressed Sparse Row format>

In [262]:
print(char_vec.vocabulary_)

{'d': 4, 'e': 5, 'v': 22, 'i': 9, 'n': 14, 'k': 11, 'r': 18, 'a': 1, 'g': 7, 'l': 12, 'x': 24, 's': 19, 'm': 13, 't': 20, ' ': 0, 'p': 16, 'h': 8, 'u': 21, 'w': 23, 'j': 10, 'o': 15, 'b': 2, 'y': 25, 'f': 6, 'c': 3, 'q': 17, 'z': 26}


### Model

In [263]:
train_score = {}
test_score = {}

In [264]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score

In [265]:
from sklearn.naive_bayes import GaussianNB

* Naive Bayes

In [266]:
model_1 = 'NavieB'

In [267]:
naive_b = GaussianNB()

In [268]:
naive_b.fit(char_name_vec.toarray(),y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [269]:
p = naive_b.predict(conv_input('Narendra Modi',char_vec).toarray())

In [270]:
predict_from_name(naive_b,'Narendra Modi',char_vec)

Name :  Narendra Modi 
 Race : indian


In [271]:
naive_b.score(char_name_vec.toarray(),y_train)

0.38528422372997323

In [272]:
naive_b.score(char_vec.transform(name_to_list(name_test)).toarray(),race_test)

0.36875712656784493

## Custom Features

In [230]:
from sklearn.feature_extraction import DictVectorizer

In [231]:
m_result = {'train_score' : {},'test_score' : {}}

In [232]:
custom_vec = DictVectorizer()

In [233]:
train_features = custom_vec.fit_transform(X_train.to_dict(orient='records')) # convert dataframe to dict like records

In [234]:
train_features

<35078x5440 sparse matrix of type '<class 'numpy.float64'>'
	with 195773 stored elements in Compressed Sparse Row format>

In [235]:
print(custom_vec.feature_names_)

['first_2=aa', 'first_2=ab', 'first_2=ac', 'first_2=ad', 'first_2=ae', 'first_2=af', 'first_2=ag', 'first_2=ah', 'first_2=ai', 'first_2=aj', 'first_2=ak', 'first_2=al', 'first_2=am', 'first_2=an', 'first_2=ao', 'first_2=ap', 'first_2=aq', 'first_2=ar', 'first_2=as', 'first_2=at', 'first_2=au', 'first_2=av', 'first_2=aw', 'first_2=ax', 'first_2=ay', 'first_2=az', 'first_2=ba', 'first_2=be', 'first_2=bh', 'first_2=bi', 'first_2=bl', 'first_2=bo', 'first_2=br', 'first_2=bu', 'first_2=by', 'first_2=ca', 'first_2=ce', 'first_2=ch', 'first_2=ci', 'first_2=cl', 'first_2=co', 'first_2=cr', 'first_2=cs', 'first_2=cu', 'first_2=cy', 'first_2=da', 'first_2=dc', 'first_2=de', 'first_2=dh', 'first_2=di', 'first_2=dj', 'first_2=dm', 'first_2=do', 'first_2=dq', 'first_2=dr', 'first_2=ds', 'first_2=du', 'first_2=dv', 'first_2=dw', 'first_2=dy', 'first_2=ea', 'first_2=eb', 'first_2=ec', 'first_2=ed', 'first_2=ee', 'first_2=ef', 'first_2=eg', 'first_2=eh', 'first_2=ei', 'first_2=ek', 'first_2=el', 'firs

* Naive Bayes

In [236]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB

In [237]:
nbm = MultinomialNB()

In [238]:
nbm.fit(train_features,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [239]:
nbm.score(train_features,y_train)

0.7485603512172871

In [240]:
nbm.score(custom_vec.transform(X_test.to_dict(orient='records')),y_test)

0.7094640820980616

* Decision Tree

In [241]:
from sklearn.tree import DecisionTreeClassifier
 
dclf = DecisionTreeClassifier()

In [242]:
def print_res(model):
    model.fit(train_features,y_train)
    print("Train score :",model.score(train_features,y_train))
    print("Test Score",model.score(custom_vec.transform(X_test.to_dict(orient='records')),y_test))

In [243]:
print_res(dclf)

Train score : 0.8918410399680712
Test Score 0.7400228050171037


* Logistic Regression

In [244]:
from sklearn.linear_model import LogisticRegression

In [245]:
print_res(LogisticRegression())

/home/master/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/master/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Train score : 0.7822851929984606
Test Score 0.7230330672748004


## Count Vectorizer on name

In [248]:
from sklearn.feature_extraction.text import CountVectorizer 

In [249]:
count_vec = CountVectorizer()

In [250]:
count_features = count_vec.fit_transform(name_train)

In [251]:
def print_util(model):
    model.fit(count_features,race_train)
    print("Train score :",model.score(count_features,race_train))
    print("Test Score",model.score(count_vec.transform(name_test),race_test))

In [252]:
print_util(LogisticRegression())

Train score : 0.8820913393009864
Test Score 0.7137970353477765


In [253]:
print_util(MultinomialNB())

Train score : 0.874308683505331
Test Score 0.6558722919042189


In [254]:
print_util(DecisionTreeClassifier())

Train score : 0.8920691031415702
Test Score 0.7157354618015963
